<a href="https://colab.research.google.com/github/Aragnzlz/SIMULACION-II/blob/main/SIMPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SIMULACION DE EVENTOS DISCRETOS CON SIMPY**

La biblioteca SimPy es una biblioteca en Python diseñada para modelar y simular sistemas de eventos discretos.
Se utiliza para simular eventos como:
* líneas de producción
* sistemas de colas
* redes de transporte
entre otros.

**CONCEPTOS CLAVE**

1. **Eventos discretos:**  el tiempo en el modelo avanza de un evento al siguiente, en lugar de ser continuo.
2. **Procesos:** cualquier acción que ocurre en el sistema simulado, se puede indentificar un proceso dentro del programa con la palabra yield.
3. **Recursos:** existen diferentes tipos de recursos.
    *   Resourse: recurso genérico
    *   Container: recurso con una cantidad específica de objetos.
    *   Store: permite almacenar objetos.
    *   PriorityResourse: se asigna prioridades a los procesos.
4. **Entorno(Enviroment):** es el núcleo de una simulación de SimPy y gestiona el tiempo simulado y todos los eventos.
5. **Eventos:** son los puntos de acción en la simulación. Cada proceso espera un evento o desencadena uno que afectará al sistema.


In [46]:
!pip install simpy

In [440]:
import simpy
import random

In [497]:
tiempos_llegada_pasajeros = []
tiempos_abordaje_metrobuses = []
tiempos_partida_metrobuses = []
tiempos_llegada_autobuses = []
tiempos_partida_autobuses = []
tiempos_espera_pasajeros = []
tiempo_partida = []
pasajeros_abordo = []
aut = [] # de autobus
n_p = [] #número de pasajero

In [509]:
# Parámetros de la simulación
TASA_LLEGADA_PASAJEROS = 0.06           # En promedio, llegan 2 pasajeros cada medio segundo
INTERVALO_LLEGADA_METROBUS = 5      # Un Metrobús cada 5 minutos
CAPACIDAD_METROBUS = 240              # Capacidad del Metrobús
TIEMPO_ABORDAJE_GRUPO = 2 / 60       # 2 segundos en minutos para abordar 8 personas
TAMANO_GRUPO_ABORDAJE = 8            # Tamaño del grupo que aborda a la vez
TIEMPO_ESPERA_METROBUS = 0.5           # 1 minuto de espera en la estación
TIEMPO_SIMULACION = 10              # Simulación de 2 horas

In [523]:
def llegada_pasajeros(env, pasajeros,tiempos_llegada,n_p):
    pasajero_id = 0
    while True:
        # Tiempo hasta la llegada de un nuevo pasajero
        yield env.timeout(random.expovariate(1 / TASA_LLEGADA_PASAJEROS))
        pasajero_id += 1
        # Añadir el pasajero a la cola de espera
        pasajeros.append(pasajero_id)
        tiempos_llegada_pasajeros.append(env.now)
        n_p.append(pasajero_id)

In [524]:
def llegada_metrobús(env, pasajeros, aut,tiempos_llegada_autobuses,tiempos_partida_metrobuses,pasajeros_abordo,tiempo_partida):
  #Llegada de los metrobuses y abordaje de pasajeros durante un minuto
    autobús_id = 0
    while True:
        # Tiempo hasta la llegada de un nuevo Metrobús
        yield env.timeout(INTERVALO_LLEGADA_METROBUS)
        autobús_id += 1
        aut.append(autobús_id)
        tiempos_llegada_autobuses.append(env.now)

        # Tiempo que el Metrobús espera en la estación para abordar pasajeros
        tiempo_final_abordaje = env.now + TIEMPO_ESPERA_METROBUS
        pasajeros_a_bordo = 0

        # Aborda en grupos de hasta 8 personas cada 2 segundos mientras dure el tiempo de espera
        while env.now < tiempo_final_abordaje and pasajeros and pasajeros_a_bordo < CAPACIDAD_METROBUS:
            grupo_abordo = min(len(pasajeros), TAMANO_GRUPO_ABORDAJE, CAPACIDAD_METROBUS - pasajeros_a_bordo)

            # Cada grupo tarda 2 segundos en abordar
            yield env.timeout(TIEMPO_ABORDAJE_GRUPO)

            # Remover el grupo de pasajeros que abordaron y contar
            for _ in range(grupo_abordo):
                tiempo_partida.append(env.now)
                pasajeros.pop(0)

            pasajeros_a_bordo += grupo_abordo


            # Partida del Metrobús
        tiempos_partida_metrobuses.append(env.now)
        pasajeros_abordo.append(pasajeros_a_bordo)

In [525]:
# Configurar el entorno de SimPy
env = simpy.Environment()
pasajeros = []  # Lista para contar los pasajeros en espera
tiempos_llegada_pasajeros = []
aut = []
tiempos_llegada_autobuses = []
tiempos_partida_metrobuses = []
pasajeros_abordo = []
tiempos_espera_pasajeros=[]
n_p = []
tiempo_partida = []

In [526]:
# Inicializar los procesos
env.process(llegada_pasajeros(env, pasajeros, tiempos_llegada_pasajeros,n_p))
env.process(llegada_metrobús(env, pasajeros,aut,tiempos_llegada_autobuses,tiempos_partida_metrobuses,pasajeros_abordo,tiempo_partida))

<Process(llegada_metrobús) object at 0x7ff29ab9f4c0>

In [527]:
# Ejecutar la simulación
env.run(until=TIEMPO_SIMULACION)

In [528]:
import numpy as np

# Definir dos arreglos
a = np.array(tiempos_llegada_pasajeros)
b = np.array(tiempo_partida)

# Obtener el tamaño del arreglo más pequeño
min_len = min(len(a), len(b))

# Recortar ambos arreglos al tamaño mínimo
a = a[:min_len]
b = b[:min_len]

tiempo_espera = [y - x for x, y in zip(a, b)]

In [529]:
# Check for the minimum length of all three lists
min_len = min(len(n_p), len(tiempos_llegada_pasajeros), len(tiempo_partida))

# Iterate only over the valid range
A = [[n_p[i], tiempos_llegada_pasajeros[i], tiempo_partida[i],tiempo_espera[i]] for i in range(min_len)]

head = ["PASAJERO", "T_LLEGADA", "T_PARTIDA","T_ESPERA"]
A.insert(0, head)

from tabulate import tabulate
# Imprimir los datos en formato de tabla
print(tabulate(A, headers="firstrow", tablefmt="fancy_grid"))

╒════════════╤═════════════╤═════════════╤════════════╕
│   PASAJERO │   T_LLEGADA │   T_PARTIDA │   T_ESPERA │
╞════════════╪═════════════╪═════════════╪════════════╡
│          1 │   0.0268915 │     5.03333 │  5.00644   │
├────────────┼─────────────┼─────────────┼────────────┤
│          2 │   0.15378   │     5.03333 │  4.87955   │
├────────────┼─────────────┼─────────────┼────────────┤
│          3 │   0.231266  │     5.03333 │  4.80207   │
├────────────┼─────────────┼─────────────┼────────────┤
│          4 │   0.232807  │     5.03333 │  4.80053   │
├────────────┼─────────────┼─────────────┼────────────┤
│          5 │   0.285499  │     5.03333 │  4.74783   │
├────────────┼─────────────┼─────────────┼────────────┤
│          6 │   0.291572  │     5.03333 │  4.74176   │
├────────────┼─────────────┼─────────────┼────────────┤
│          7 │   0.292929  │     5.03333 │  4.7404    │
├────────────┼─────────────┼─────────────┼────────────┤
│          8 │   0.298685  │     5.03333 │  4.73

In [530]:
min_len = min(len(aut), len(tiempos_llegada_autobuses), len(tiempos_partida_metrobuses))
A = [[aut[i], tiempos_llegada_autobuses[i], tiempos_partida_metrobuses[i],pasajeros_abordo[i]] for i in range(min_len)]

head = ["METROBÚS", "T_LLEGADA", "T_PARTIDA", "P. ABORDO"]
A.insert(0, head)

from tabulate import tabulate

# Imprimir los datos en formato de tabla
print(tabulate(A, headers="firstrow", tablefmt="fancy_grid"))

╒════════════╤═════════════╤═════════════╤═════════════╕
│   METROBÚS │   T_LLEGADA │   T_PARTIDA │   P. ABORDO │
╞════════════╪═════════════╪═════════════╪═════════════╡
│          1 │           5 │         5.5 │         105 │
╘════════════╧═════════════╧═════════════╧═════════════╛


In [531]:
def simulacion():
# Configurar el entorno de SimPy
    env = simpy.Environment()
    pasajeros = []  # Lista para contar los pasajeros en espera
    tiempos_llegada_pasajeros = []
    aut = []
    tiempos_llegada_autobuses = []
    tiempos_partida_metrobuses = []
    pasajeros_abordo = []
    tiempos_espera_pasajeros=[]
    n_p = []
    tiempo_partida = []
    tiempo_espera = []

    # Inicializar los procesos
    env.process(llegada_pasajeros(env, pasajeros, tiempos_llegada_pasajeros,n_p))
    env.process(llegada_metrobús(env, pasajeros,aut,tiempos_llegada_autobuses,tiempos_partida_metrobuses,pasajeros_abordo,tiempo_partida))

    # Ejecutar la simulación
    env.run(until=TIEMPO_SIMULACION)

In [536]:
n = 2
media_espera =[]
for i in range(n):
  simulacion()
  a = np.array(tiempos_llegada_pasajeros)
  b = np.array(tiempo_partida)

  # Obtener el tamaño del arreglo más pequeño
  min_len = min(len(a), len(b))

  # Recortar ambos arreglos al tamaño mínimo
  a = a[:min_len]
  b = b[:min_len]

  tiempo_espera = [y - x for x, y in zip(a, b)]
  print(tiempo_partida)



[5.033333333333333, 5.033333333333333, 5.033333333333333, 5.033333333333333, 5.033333333333333, 5.033333333333333, 5.033333333333333, 5.033333333333333, 5.066666666666666, 5.066666666666666, 5.066666666666666, 5.066666666666666, 5.066666666666666, 5.066666666666666, 5.066666666666666, 5.066666666666666, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.133333333333333, 5.133333333333333, 5.133333333333333, 5.133333333333333, 5.133333333333333, 5.133333333333333, 5.133333333333333, 5.133333333333333, 5.166666666666666, 5.166666666666666, 5.166666666666666, 5.166666666666666, 5.166666666666666, 5.166666666666666, 5.166666666666666, 5.166666666666666, 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.199999999999999, 5.2333333333333325, 5.2333333333333325, 5.2333333333333325, 5.2333333333333325, 5.2333333333333325, 5.2333333333333325, 5.2333333333333325, 5.2333333333333325, 5.266666666666666, 5.266666666666666, 5

In [537]:
print(media_espera)

[]


In [538]:
# Definir dos arreglos
a = np.array(tiempos_llegada_pasajeros)
b = np.array(tiempo_partida)

# Obtener el tamaño del arreglo más pequeño
min_len = min(len(a), len(b))

# Recortar ambos arreglos al tamaño mínimo
a = a[:min_len]
b = b[:min_len]

tiempo_espera = [y - x for x, y in zip(a, b)]

In [522]:
# Check for the minimum length of all three lists
min_len = min(len(n_p), len(tiempos_llegada_pasajeros), len(tiempo_partida))

# Iterate only over the valid range
A = [[n_p[i], tiempos_llegada_pasajeros[i], tiempo_partida[i],tiempo_espera[i]] for i in range(min_len)]

head = ["PASAJERO", "T_LLEGADA", "T_PARTIDA","T_ESPERA"]
A.insert(0, head)

from tabulate import tabulate
# Imprimir los datos en formato de tabla
print(tabulate(A, headers="firstrow", tablefmt="fancy_grid"))

╒════════════╤═════════════╤═════════════╤════════════╕
│   PASAJERO │   T_LLEGADA │   T_PARTIDA │   T_ESPERA │
╞════════════╪═════════════╪═════════════╪════════════╡
│          1 │    0.068438 │     5.03333 │   4.9649   │
├────────────┼─────────────┼─────────────┼────────────┤
│          2 │    0.170286 │     5.03333 │   4.86305  │
├────────────┼─────────────┼─────────────┼────────────┤
│          3 │    0.216106 │     5.03333 │   4.81723  │
├────────────┼─────────────┼─────────────┼────────────┤
│          4 │    0.264428 │     5.03333 │   4.76891  │
├────────────┼─────────────┼─────────────┼────────────┤
│          5 │    0.277992 │     5.03333 │   4.75534  │
├────────────┼─────────────┼─────────────┼────────────┤
│          6 │    0.401375 │     5.03333 │   4.63196  │
├────────────┼─────────────┼─────────────┼────────────┤
│          7 │    0.436918 │     5.03333 │   4.59642  │
├────────────┼─────────────┼─────────────┼────────────┤
│          8 │    0.446318 │     5.03333 │   4.5